In [1]:
# !pip install neuralforecast datasetsforecast

In [1]:
# pip install horovod==0.26.1

In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
from typing import List
from datasetsforecast.long_horizon import LongHorizon
import pandas as pd
import numpy as np
import torch
from typing import Optional

from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.auto import tqdm

from dataset import LongHorizonUnivariateDataModule, LongHorizonUnivariateDataset
import pytorch_lightning as pl


In [47]:
%%time

df = pd.read_excel('data/downloads/emhires_pv/EMHIRES_PVGIS_TSh_CF_n2_19862015_reformatt.xlsx')
df.drop('SE33', axis=1, inplace=True)
df['time_step'] = pd.date_range(start='1986-01-01', periods=len(df), freq='h')
df = df.rename({'time_step': 'ds'}, axis=1).set_index('ds')
df.to_parquet('data/emhires/pv_n2.parquet')

CPU times: user 18min 19s, sys: 9.43 s, total: 18min 28s
Wall time: 18min 28s


In [25]:
%%time

df = pd.read_excel('data/downloads/emhires_pv/EMHIRES_PVGIS_TSh_CF_n2_19862015_reformatt.xlsx')
df.drop('SE33', axis=1, inplace=True)
df['time_step'] = pd.date_range(start='1986-01-01', periods=len(df), freq='h')
df = df.rename({'time_step': 'ds'}, axis=1).set_index('ds')
df.to_parquet('data/emhires/pv_n2.parquet')

In [44]:
df.to_csv('test_pd.csv')

In [45]:
%%time

a = pd.read_parquet('test_pd.parquet')

CPU times: user 1.42 s, sys: 3.46 s, total: 4.88 s
Wall time: 392 ms


In [46]:
a

,AT11,AT21,AT12,AT31,AT32,AT22,AT33,AT34,AT13,BE21,...,UKI5UKI6,UKG2,UKM3,UKE3,UKJ2,UKC1,UKG3,UKL1,UKE4,ds
time_step,,,,,,,,,,,,,,,,,,,,,
1986-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1986-01-01 00:00:00
1986-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1986-01-01 01:00:00
1986-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1986-01-01 02:00:00
1986-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1986-01-01 03:00:00
1986-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1986-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-12-31 19:00:00
2015-12-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-12-31 20:00:00
2015-12-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-12-31 21:00:00


In [3]:
def collate_fn_flat_deal(batch):
    out = {}
    for b in batch:
        for k, bv in b.items():
            v = out.get(k, [])
            v.append(bv)
            out[k] = v
            
    for k,v in out.items():
        v = np.concatenate(v)
        if type(v[0]) not in [np.str_, pd.Timestamp]:
            v = torch.as_tensor(v)
        out[k] = v
    return out

class LongHorizonUnivariateDataModule(pl.LightningDataModule):
    def __init__(self, 
                 name: str = 'ETTm2', 
                 train_batch_size: int = 128, 
                 eval_batch_size: int = None,
                 num_workers: int = 4,
                 persistent_workers: bool = True,
                 horizon_length: int = 720,
                 history_length: int = 720,
                 split_proportions: List[float] = [0.6, 0.2, 0.2]
                ):
        super().__init__()
        self.name = name
        self.train_batch_size = train_batch_size
        self.eval_batch_size = train_batch_size
        if eval_batch_size is not None:
            self.eval_batch_size = eval_batch_size
        self.num_workers = num_workers
        self.persistent_workers = persistent_workers
        self.history_length = history_length
        self.horizon_length = horizon_length
        self.split_proportions = np.array(split_proportions).cumsum()
        
        assert self.split_proportions[-1] == 1, "Split proportions must sum up to 1"

    def prepare_data(self):
        LongHorizon.load(directory='./data', group='ETTm2')

    def setup(self, stage: str):
        # Assign train/val datasets for use in dataloaders
        if stage == "fit":
            self.train_dataset = LongHorizonUnivariateDataset(name=self.name, split='train', 
                                                              split_start=0.0,
                                                              split_end=self.split_proportions[0],
                                                              horizon_length=self.horizon_length,
                                                              history_length = self.history_length)
            self.val_dataset = LongHorizonUnivariateDataset(name=self.name, split='val', 
                                                            split_start=self.split_proportions[0],
                                                            split_end=self.split_proportions[1],
                                                            horizon_length=self.horizon_length,
                                                            history_length = self.history_length)
        # Assign test dataset for use in dataloader(s)
        if stage == "test":
            self.test_dataset = LongHorizonUnivariateDataset(name=self.name, split='test', 
                                                             split_start=self.split_proportions[1],
                                                             split_end=self.split_proportions[2],
                                                             horizon_length=self.horizon_length,
                                                             history_length = self.history_length)
        if stage == "predict":
            self.predict_dataset = LongHorizonUnivariateDataset(name=self.name, split='test', 
                                                                split_start=self.split_proportions[1],
                                                                split_end=self.split_proportions[2],
                                                                horizon_length=self.horizon_length,
                                                                history_length = self.history_length)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.train_batch_size, 
                          shuffle=True, pin_memory=True, 
                          persistent_workers=self.persistent_workers,
                          num_workers=self.num_workers, collate_fn=collate_fn_flat_deal)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.eval_batch_size, 
                          shuffle=False, pin_memory=True, 
                          persistent_workers=self.persistent_workers,
                          num_workers=self.num_workers, collate_fn=collate_fn_flat_deal)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.eval_batch_size,
                          shuffle=False, pin_memory=True, 
                          persistent_workers=self.persistent_workers,
                          num_workers=self.num_workers, collate_fn=collate_fn_flat_deal)

    def predict_dataloader(self):
        return DataLoader(self.predict_dataset, batch_size=self.eval_batch_size, 
                          shuffle=False, pin_memory=True, 
                          persistent_workers=self.persistent_workers,
                          num_workers=self.num_workers, collate_fn=collate_fn_flat_deal)

In [4]:
dm = LongHorizonUnivariateDataModule(train_batch_size=128)
dm.prepare_data()
dm.setup(stage='fit')
dm.setup(stage='test')

In [5]:
dm.split_proportions

array([0.6, 0.8, 1. ])

In [6]:
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()
test_loader = dm.test_dataloader()

In [9]:
train_loader.dataset.time_features.shape

(1, 34560, 3)

In [100]:
for b in tqdm(train_loader):
    pass

  0%|          | 0/1812 [00:00<?, ?it/s]

In [104]:
b

{'target': tensor([[-0.7275, -0.7085, -0.6896,  ..., -1.2778, -1.2778, -1.2778],
         [-0.7085, -0.6896, -0.6896,  ..., -1.2778, -1.2778, -1.2778],
         [-0.6896, -0.6896, -0.7085,  ..., -1.2778, -1.2778, -1.2778],
         ...,
         [-0.8413, -0.7465, -0.6895,  ..., -1.5813, -1.5813, -1.5813],
         [-0.7465, -0.6895, -0.6327,  ..., -1.5813, -1.5813, -1.5623],
         [-0.6895, -0.6327, -0.6137,  ..., -1.5813, -1.5623, -1.5623]],
        dtype=torch.float64),
 'history': tensor([[-2.0367, -2.0556, -2.0556,  ..., -0.7654, -0.7275, -0.7275],
         [-2.0556, -2.0556, -2.0936,  ..., -0.7275, -0.7275, -0.7275],
         [-2.0556, -2.0936, -2.0936,  ..., -0.7275, -0.7275, -0.7085],
         ...,
         [-1.8659, -1.8659, -1.8849,  ..., -1.1070, -1.0121, -0.9362],
         [-1.8659, -1.8849, -1.9038,  ..., -1.0121, -0.9362, -0.8413],
         [-1.8849, -1.9038, -1.9038,  ..., -0.9362, -0.8413, -0.7465]],
        dtype=torch.float64),
 'series_id': tensor([6, 6, 6, 6, 6, 

In [101]:
for b in tqdm(val_loader):
    pass

  0%|          | 0/591 [00:00<?, ?it/s]

In [102]:
for b in tqdm(test_loader):
    pass

  0%|          | 0/591 [00:00<?, ?it/s]

In [88]:
ETTm2 = LongHorizonUnivariateDataset(name='ETTm2', split='train', split_start=0, split_end=0.6)

In [90]:
len(ETTm2.df)

34560

In [31]:
ETTm2.df.index.max()

Timestamp('2017-02-01 00:00:00')

In [18]:
len(ETTm2.df)

57600

In [20]:
(12240-720) / 57600

0.2

In [13]:
ETTm2[2*ETTm2.num_windows+ETTm2.num_windows-1]

{'target': array([], shape=(1, 0), dtype=float64),
 'history': array([[-0.07068447]]),
 'series_id': array([3])}

In [9]:
def collate_fn_flat_deal(batch):
    out = {}
    for b in batch:
        for k, bv in b.items():
            v = out.get(k, [])
            v.append(bv)
            out[k] = v
            
    for k,v in out.items():
        v = np.concatenate(v)
        if type(v[0]) not in [np.str_, pd.Timestamp]:
            v = torch.as_tensor(v)
        out[k] = v
    return out

dl = DataLoader(ETTm2, batch_size=512, 
                          shuffle=True, pin_memory=True, 
                          persistent_workers=True,
                          num_workers=4, collate_fn=collate_fn_flat_deal)

In [10]:
for b in tqdm(dl):
    
    a=1

  0%|          | 0/453 [00:00<?, ?it/s]

In [11]:
b['history'].shape

torch.Size([423, 720])

In [69]:
Y_df.ds = pd.to_datetime(Y_df.ds)
# Y_df = Y_df.pivot(index='ds', columns='unique_id', values='y')

Y_df

,unique_id,ds,y
0,HUFL,2016-07-01 00:00:00,-0.041413
1,HUFL,2016-07-01 00:15:00,-0.185467
2,HUFL,2016-07-01 00:30:00,-0.257495
3,HUFL,2016-07-01 00:45:00,-0.577510
4,HUFL,2016-07-01 01:00:00,-0.385501
...,...,...,...
403195,OT,2018-02-20 22:45:00,-1.581325
403196,OT,2018-02-20 23:00:00,-1.581325
403197,OT,2018-02-20 23:15:00,-1.581325
403198,OT,2018-02-20 23:30:00,-1.562328


unique_id                  HUFL      HULL      LUFL      LULL      MUFL  \
  ds                                                                      
y 2016-07-01 00:00:00 -0.041413  0.040104  0.695804  0.434430 -0.599211   
  2016-07-01 00:15:00 -0.185467 -0.214450  0.434685  0.428168 -0.658068   
  2016-07-01 00:30:00 -0.257495 -0.378215  0.698168  0.428168 -0.728020   
  2016-07-01 00:45:00 -0.577510 -0.669474  0.698168  0.423087 -0.834580   
  2016-07-01 01:00:00 -0.385501 -0.469220  0.698168  0.423087 -0.753474   
...                         ...       ...       ...       ...       ...   
  2018-02-20 22:45:00 -1.377739 -1.742855 -2.475240 -0.070684 -0.514900   
  2018-02-20 23:00:00 -1.409741 -1.724610 -2.743237 -0.070684 -0.438540   
  2018-02-20 23:15:00 -1.433814 -1.706366 -2.743237 -0.070684 -0.476750   
  2018-02-20 23:30:00 -1.305712 -1.633605 -2.743237 -0.070684 -0.470342   
  2018-02-20 23:45:00 -1.513770 -1.742855 -2.736360 -0.070684 -0.529199   

unique_id                  MULL        OT  
  ds                                       
y 2016-07-01 00:00:00 -0.393536  1.018032  
  2016-07-01 00:15:00 -0.659338  0.980124  
  2016-07-01 00:30:00 -0.978366  0.904223  
  2016-07-01 00:45:00 -0.606111  0.885226  
  2016-07-01 01:00:00 -0.996218  0.885226  
...                         ...       ...  
  2018-02-20 22:45:00 -1.253094 -1.581325  
  2018-02-20 23:00:00 -1.120193 -1.581325  
  2018-02-20 23:15:00 -1.129119 -1.581325  
  2018-02-20 23:30:00 -1.102341 -1.562328  
  2018-02-20 23:45:00 -1.173419 -1.562328  

[57600 rows x 7 columns]

In [39]:
Y_df

y
unique_id ds                           
HUFL      2016-07-01 00:00:00 -0.041413
          2016-07-01 00:15:00 -0.185467
          2016-07-01 00:30:00 -0.257495
          2016-07-01 00:45:00 -0.577510
          2016-07-01 01:00:00 -0.385501
...                                 ...
OT        2018-02-20 22:45:00 -1.581325
          2018-02-20 23:00:00 -1.581325
          2018-02-20 23:15:00 -1.581325
          2018-02-20 23:30:00 -1.562328
          2018-02-20 23:45:00 -1.562328

[403200 rows x 1 columns]

In [11]:
# from neuralforecast import TimeSeriesDataset

from neuralforecast.tsdataset import TimeSeriesDataset

In [24]:
ds, uids, last_dates, ds_sort  = TimeSeriesDataset.from_df(Y_df)

In [32]:
ds[0]['temporal']

torch.Size([2, 57600])

In [33]:
len(ds)

7